In [10]:
import Pkg; Pkg.activate("..")

  Activating project at `~/SAFT_ML`


In [11]:
using CSV, DataFrames, DelimitedFiles, Clapeyron, PyCall
import PyPlot; const plt = PyPlot ;

In [16]:
validation_data_saft_vr_mie = CSV.read("saft_vr_mie_validation_data.csv",DataFrame)
num_rows = nrow(validation_data_saft_vr_mie)

rename!(validation_data_saft_vr_mie,:sat_t_K_clapeyron=>:sat_temp_strings, :sat_p_MPa_clapeyron=>:sat_pressure_strings,
        :sat_vliq_m3_per_mol_clapeyron=>:sat_vol_liq_strings,:sat_vvap_m3_per_mol_clapeyron=>:sat_vol_vap_strings)

# Define the column names
array_cols_to_create = [:sat_t_K_clapeyron, :sat_p_MPa_clapeyron, :sat_vliq_m3_per_mol_clapeyron, :sat_vvap_m3_per_mol_clapeyron]
string_columns = [:sat_temp_strings, :sat_pressure_strings, :sat_vol_liq_strings, :sat_vol_vap_strings]

# Initialize the columns with zeros
for col in array_cols_to_create
    validation_data_saft_vr_mie[!, col] .= fill([0.0], num_rows)
end

# Process and assign the values to the DataFrame
for (col, str_col) in zip(array_cols_to_create, string_columns)
    for i = 1:num_rows
        string_sq_brackets_removed = replace(validation_data_saft_vr_mie[i, str_col], r"[][]" => "")
        vector_of_floats = [parse(Float64, val) for val in split(string_sq_brackets_removed, ",")]
        validation_data_saft_vr_mie[i, col] = vector_of_floats
    end
end

# Remove the string columns
select!(validation_data_saft_vr_mie, Not(string_columns...)) ;

# training_data